In [1]:
from builder import AgentBuilder

In [2]:
agent = await AgentBuilder.build(
    name="WeatherAgent",
    user_id="1234",
    tool_names=["location", "weather"],
    user_name="id4thomas"
)

In [3]:
query="내 현재 위치와 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘."
# query="서울 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘. 도시 명칭은 반환값 그대로 사용해"
# query="도구를 사용해서 서울 날씨 알려줘. 도구의 결과도 json 그대로 알려줘"

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": query}
        ]
    }
]

# Invoke

In [4]:
# ainvoke
result = await agent.ainvoke({"messages": messages})

In [5]:
for msg_i,msg in enumerate(result['messages']):
    print(f"MSG {msg_i} - {type(msg)}")
    print(f"\t{repr(msg.content)}")
    if msg.type=='ai':
        print(f"\ttools:{msg.tool_calls}")

MSG 0 - <class 'langchain_core.messages.human.HumanMessage'>
	[{'type': 'text', 'text': '내 현재 위치와 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘.'}]
MSG 1 - <class 'langchain_core.messages.ai.AIMessage'>
	'알겠습니다. 먼저 현재 위치를 확인하고, 그 주변 도시들을 확인한 후 날씨 정보를 제공해드리겠습니다.\n'
	tools:[{'name': 'get_user_location', 'args': {}, 'id': 'chatcmpl-tool-8d912d6ad0a3d982', 'type': 'tool_call'}]
MSG 2 - <class 'langchain_core.messages.tool.ToolMessage'>
	[{'type': 'text', 'text': '{"name":"Seoul","coordinate":{"lat":37.5665,"lon":126.978}}', 'id': 'lc_62771627-2dae-4501-93e7-b0f83be28010'}]
MSG 3 - <class 'langchain_core.messages.ai.AIMessage'>
	'현재 위치는 서울입니다. 주변 도시들을 확인해보겠습니다.\n\n'
	tools:[{'name': 'get_nearby_cities', 'args': {'city_name': 'Seoul'}, 'id': 'chatcmpl-tool-974870f8e87e4f0f', 'type': 'tool_call'}]
MSG 4 - <class 'langchain_core.messages.tool.ToolMessage'>
	[{'type': 'text', 'text': '[{"name":"Seongnam","coordinate":{"lat":37.42,"lon":127.1267}},{"name":"Suwon","coordinate":{"lat":37.2636,"lon":127.0286}},{"na

# Streaming

In [6]:
result_stream = agent.astream(
    {"messages": messages},
    stream_mode="updates"
)

In [7]:
chunk_i=0
async for chunk in result_stream:
    chunk_i+=1
    print(f"Chunk{chunk_i}")
    for node, data in chunk.items():
        last_msg = data["messages"][-1]

        if node == "model":
            print("model node")
            content_blocks = last_msg.content_blocks or []
            tool_calls = [b for b in content_blocks if b.get("type") == "tool_call"]

            if tool_calls:
                for tc in tool_calls:
                    print("\t",{"type": "tool_call", "tool": tc["name"], "args": tc["args"]})
            else:
                text = "".join(b["text"] for b in content_blocks if b.get("type") == "text")
                if text:
                    print("\t",{"type": "ai_message", "content": text})

        elif node == "tools":
            print("tools node")
            content_blocks = last_msg.content_blocks or []
            result = "".join(b["text"] for b in content_blocks if b.get("type") == "text")
            print("\t",{"type": "tool_result", "content": result})

Chunk1
model node
	 {'type': 'tool_call', 'tool': 'get_user_location', 'args': {}}
Chunk2
tools node
	 {'type': 'tool_result', 'content': '{"name":"Seoul","coordinate":{"lat":37.5665,"lon":126.978}}'}
Chunk3
model node
	 {'type': 'tool_call', 'tool': 'get_nearby_cities', 'args': {'city_name': 'Seoul'}}
Chunk4
tools node
	 {'type': 'tool_result', 'content': '[{"name":"Seongnam","coordinate":{"lat":37.42,"lon":127.1267}},{"name":"Suwon","coordinate":{"lat":37.2636,"lon":127.0286}},{"name":"Changwon","coordinate":{"lat":35.228,"lon":128.6811}}]'}
Chunk5
model node
	 {'type': 'tool_call', 'tool': 'get_current_weather', 'args': {'city_name': 'Seoul'}}
	 {'type': 'tool_call', 'tool': 'get_current_weather', 'args': {'city_name': 'Seongnam'}}
	 {'type': 'tool_call', 'tool': 'get_current_weather', 'args': {'city_name': 'Suwon'}}
	 {'type': 'tool_call', 'tool': 'get_current_weather', 'args': {'city_name': 'Changwon'}}
Chunk6
tools node
	 {'type': 'tool_result', 'content': '{"time":"20260219-00:0